**EDA**

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kritanjalijain/amazon-reviews")
path += "/test.csv"
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/amazon-reviews/test.csv


In [34]:
import pandas as pd
from tabulate import tabulate

df = pd.read_csv(path)
df.columns = ['Label', 'Review', "Detailed Review"]

In [35]:
df.isnull().sum()

,0
Label,0
Review,24
Detailed Review,0


In [95]:
null_reviews = df[df['Review'].isnull()]
longest_review_null = null_reviews["Detailed Review"].str.len().idxmax()
print(null_reviews.loc[longest_review_null, "Detailed Review"])
print("Length of the longest review:", len(null_reviews.loc[longest_review_null, "Detailed Review"]))

Some good rocking moments here and there (especially on "North Berwick Witch Trials" and "Oro the Manslayer"), but other than that Cathedral's style of Sabbath-isms combined with a sort of Southern Hard Rock groove can easily grow tired. "The Garden" also has its moments and strives to be different, but at 27 minutes long, it is just too long and sounds like an experiment gone wrong. Listening to some parts of this record reminded me of the funny tough-guy attitude brought by bands like Crowbar, which does not surprise considering who the producer is here. Cathedral rocks hard, but not in a way to maintain someone's interest for more than 5 minutes.
Length of the longest review: 657


In [99]:
df['Review'] = df['Review'].fillna(df['Detailed Review'])

,0
Label,0
Review,0
Detailed Review,0


In [36]:
# prompt: find longest string in df['Review']

longest_review = df['Review'].str.len().idxmax()
print(df.loc[longest_review, 'Review'])
print("Length of the longest review:", len(df.loc[longest_review, 'Review']))

[Two and a half stars out of five] Average from the word go; musclehead Bill Goldberg, the ex-Pro DLm ends his movie career here
Length of the longest review: 128


In [42]:
longest_review = df['Detailed Review'].str.len().idxmax()
print(df.loc[longest_review, 'Detailed Review'])
print("Length of the longest review:", len(df.loc[longest_review, 'Detailed Review']))

Originally had a Cuisinart thermal, 12 cup. This coffee maker was $100 and I went through three of them in two years. The thermos was great but the coffee pot is designed terrible. I figured "how could I go wrong with a Mr. Coffee?" They were the first and should be reliable. I bought their Thermal Pot for about $60 and it lasted two months. One pot of coffee took almost thirty minutes to complete. The steam that came out of the top was unreal! The clock stopped working due to the excessive moisture and I took it back and got another one from Walmart. The second one worked great for two weeks and then started with the excessive steam problem again, although not as bad as the first pot. The beeper is almost too quiet. Some reviewers have said it is too loud. Not mine! I am really disappoint that Mr. Coffee would sell a product that is this inconsistent in performace. Their marketing director must have been a castaway from Microsoft. Apparently it is a good pot if you get one that works 

In [38]:
# prompt: average length of df[DetailedReview']

average_length = df['Detailed Review'].str.len().mean()
print("Average length of detailed reviews:", average_length)

Average length of detailed reviews: 404.8999022497556


In [40]:
# prompt: 75th quantile of df['Detailed Review']

quantile_75 = df['Detailed Review'].str.len().quantile(0.75)
print("75th quantile of detailed review length:", quantile_75)

75th quantile of detailed review length: 565.0


In [39]:
# prompt: average length of df[DetailedReview']

average_length = df['Review'].str.len().mean()
print("Average length of detailed reviews:", average_length)

Average length of detailed reviews: 24.530768173010813


In [43]:
print(df['Review'].str.len().quantile(0.75))

32.0


Token length for Detailed review = 600 \\
Token length for Review = 50

**Tokenizer**

In [44]:
D_size = 600
R_size = 50

In [104]:
import regex as re

GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

class Tokenizer:
    def __init__(self):

        self.lookup = {}
        self.merge = False

    def train(self, train:str, vocab_size:int, verbose=False):

        if len(train) > vocab_size:
          self.merge = True

        token = re.compile(GPT4_SPLIT_PATTERN)
        token_list = re.findall(token, train)

        if verbose:
          (i.encode("UTF-8") for i in token_list)
        print(token_list)

    def encode(self, token):
        pass


In [105]:
niga = Tokenizer()

for i in df['Review']:
  niga.train(i, R_size, verbose=True)

Streaming output truncated to the last 5000 lines.
['Disjointed']
['Easy', ' read', ' on', ' a', ' huge', ' subject']
['Great', ' Book', '!']
['AN', ' EXCELLENT', ' BOOK']
['A', ' pretty', ' good', ' book']
['As', ' good', ' as', ' a', ' history', ' book', ' can', ' be']
['My', ' Absolute', ' Favorite', '!']
['Sloppy', ' Kindle', ' conversion', ' ruins', ' the', ' experience']
['Astonishingly', ' Good']
['The', ' Best', ' Book', ' Ever']
['5', ' stars', ' and', ' counting']
['A', ' must', ' read']
['Tough', ' reading']
['keeps', ' you', ' guessing']
['A', ' Tenor', ' to', ' the', ' Music', ' ,', 'not', ' to', ' the', ' Palaeontology', '.']
['Jose', ' Cura', ' the', ' Dramatic', ' Tenor']
['very', ' cool']
['WHAT', '??????????????']
['BLAHHHH']
['I', ' learned', ' something', ' new']
['More', ' music', ' for', ' insects']
['Great', ' Book', '!']
['Don', "'t", ' believe', ' everything', ' you', ' read']
['Terrific', ' music', ',', ' inspiring', ' performance']
['Works', ' fine']
['This',